# Введение в MapReduce модель на Python


In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.470033109395096),
 (1, 1.470033109395096),
 (2, 1.470033109395096),
 (3, 1.470033109395096),
 (4, 1.470033109395096)]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('what', 10)]),
 (1, [('a', 2), ('is', 18), ('it', 18)])]

## TeraSort

In [ ]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.04505896562137246),
   (None, 0.052517753518649624),
   (None, 0.09592507753022173),
   (None, 0.21686007660697104),
   (None, 0.2381214076576984),
   (None, 0.2687138588594764),
   (None, 0.33293926271632757),
   (None, 0.3483926605165497),
   (None, 0.42064477844919534),
   (None, 0.44704937026851743),
   (None, 0.4618798499706903),
   (None, 0.4865439527238299),
   (None, 0.49209569409335463)]),
 (1,
  [(None, 0.5150706072908666),
   (None, 0.5206213577105443),
   (None, 0.5364673088375026),
   (None, 0.6526555707220714),
   (None, 0.6540514556416043),
   (None, 0.7021728573914745),
   (None, 0.7121054975283118),
   (None, 0.715942462943986),
   (None, 0.7356090736894821),
   (None, 0.7413721021894909),
   (None, 0.8375244864609163),
   (None, 0.8439351567571636),
   (None, 0.8458890411775093),
   (None, 0.869334797786821),
   (None, 0.8693502088658266),
   (None, 0.9183935469203568),
   (None, 0.9300470008378819)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [ ]:
def MAP_MAX(_, row: NamedTuple):
  yield("max", row)

def REDUCE_MAX(max_str: str, rows: Iterator[NamedTuple]):
  max_value_user = rows[0]
  for row in rows:
    if(row.social_contacts > max_value_user.social_contacts):
      max_value_user = row
  yield (max_str, max_value_user)


def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX)
output = list(output)
output



[('max', User(id=3, age=33, social_contacts=800, gender='female'))]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [ ]:
def MAP_AVERAGE(_, row: NamedTuple):
  yield("average", row)

def REDUCE_AVERAGE(average_str: str, rows: Iterator[NamedTuple]):
  count = 0
  summa = 0
  for row in rows:
    summa += row.social_contacts
    count += 1
  yield (average_str, summa/count)


def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP_AVERAGE, REDUCE_AVERAGE)
output = list(output)
output

[('average', 390.0)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [ ]:
def groupbykey(iterable):
    # Сортировка по id
    iterable = sorted(iterable, key=lambda user: user[0])
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

output = MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX)
output = list(output)
print(f"max: {output}")

output = MapReduce(RECORDREADER, MAP_AVERAGE, REDUCE_AVERAGE)
output = list(output)
print(f"average: {output}")

max: [('max', User(id=3, age=33, social_contacts=800, gender='female'))]
average: [('average', 390.0)]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [ ]:
input_collection = [
    User(id=0, age=54, gender="male", social_contacts=2000),
    User(id=0, age=25, gender="female", social_contacts=240),
    User(id=1, age=27, gender="male", social_contacts=642),
    User(id=2, age=22, gender="female", social_contacts=123),
    User(id=2, age=35, gender="female", social_contacts=247),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=3, age=15, gender="female", social_contacts=73),
]

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

maps = 3
reducers = 2


def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for userindex, user in enumerate(split):
            yield (userindex, user)

    split_size = int(np.ceil(len(input_collection) / maps))
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i : i + split_size])


def MAP_UNIQUE(_, user: str):
    yield (user.id, user)

def REDUCE_UNIQUE(user_id: int, users: Iterator[int]):
    yield users[0]


partitioned_output = MapReduceDistributed(
    INPUTFORMAT, MAP_UNIQUE, REDUCE_UNIQUE, COMBINER=None
)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]
partitioned_output


7 key-value pairs were sent over a network.


[(0,
  [User(id=0, age=54, social_contacts=2000, gender='male'),
   User(id=2, age=22, social_contacts=123, gender='female')]),
 (1,
  [User(id=1, age=27, social_contacts=642, gender='male'),
   User(id=3, age=31, social_contacts=521, gender='male')])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [ ]:
# Предикатом C в данном случае является проверка параметра gender
def MAP_SELECT(_, row: NamedTuple):
    if row.gender == "male":
        yield (row, row)


def REDUCE_SELECT(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP_SELECT, REDUCE_SELECT)
output = list(output)
output

[(User(id=0, age=54, social_contacts=2000, gender='male'),
  [User(id=0, age=54, social_contacts=2000, gender='male')]),
 (User(id=1, age=27, social_contacts=642, gender='male'),
  [User(id=1, age=27, social_contacts=642, gender='male')]),
 (User(id=3, age=31, social_contacts=521, gender='male'),
  [User(id=3, age=31, social_contacts=521, gender='male')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [ ]:
input_collection_2 = [
    dict(id=0, age=54, gender="male", social_contacts=20),
    dict(id=0, age=25, gender="female", social_contacts=240),
    dict(id=1, age=27, gender="male", social_contacts=642),
    dict(id=2, age=16, gender="male", social_contacts=123),
    dict(id=2, age=35, gender="female", social_contacts=247),
    dict(id=3, age=32, gender="male", social_contacts=521),
    dict(id=3, age=32, gender="male", social_contacts=753),
]

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )

def MAP_PROJECTION(_, iterable):# делаем проекцию по S
  S = ["age", "gender"]
  filtered_data = [{key: item[key] for key in S if key in item} for item in input_collection_2]
  for data in filtered_data:
    yield (data["age"], data)

def REDUCE_PROJECTION(row: str, rows: Iterator[NamedTuple]):# убираем повторки
  unique_rows = set()
  for row_data in rows:
      row_tuple = tuple(row_data.items())
      unique_rows.add(row_tuple)

  return [{key: value for key, value in row_tuple} for row_tuple in unique_rows]

def RECORDREADER():
    return [(u["id"], u) for u in input_collection_2]

output = MapReduce(RECORDREADER, MAP_PROJECTION, REDUCE_PROJECTION)
output = list(output)
output

[{'age': 54, 'gender': 'male'},
 {'age': 25, 'gender': 'female'},
 {'age': 27, 'gender': 'male'},
 {'age': 16, 'gender': 'male'},
 {'age': 35, 'gender': 'female'},
 {'age': 32, 'gender': 'male'}]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [ ]:
from typing import Iterator

input_collection_a = [
    User(id=1, age=54, gender="male", social_contacts=20),
    User(id=2, age=25, gender="female", social_contacts=240),
    User(id=3, age=27, gender="male", social_contacts=642),
    User(id=4, age=16, gender="male", social_contacts=123),
    User(id=5, age=35, gender="female", social_contacts=247),
]

input_collection_b = [
    User(id=5, age=45, gender="female", social_contacts=247),
    User(id=6, age=31, gender="male", social_contacts=521),
    User(id=7, age=32, gender="female", social_contacts=753),
]


def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )


def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]

def MAP_UNION(_, row: NamedTuple):
    yield(row.id, row)

# Даже если у ключа два значения, то всё равно возращаем (t, t)
def REDUCE_UNION(row: str, rows: Iterator[NamedTuple]):
    for elem in rows:
      yield(row, elem)


output = MapReduce(RECORDREADER, MAP_UNION, REDUCE_UNION)
output = list(output)
output

[(1, User(id=1, age=54, social_contacts=20, gender='male')),
 (2, User(id=2, age=25, social_contacts=240, gender='female')),
 (3, User(id=3, age=27, social_contacts=642, gender='male')),
 (4, User(id=4, age=16, social_contacts=123, gender='male')),
 (5, User(id=5, age=35, social_contacts=247, gender='female')),
 (5, User(id=5, age=45, social_contacts=247, gender='female')),
 (6, User(id=6, age=31, social_contacts=521, gender='male')),
 (7, User(id=7, age=32, social_contacts=753, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [ ]:
input_collection_a = [
    User(id=1, age=54, gender="male", social_contacts=20),
    User(id=2, age=25, gender="female", social_contacts=240),
    User(id=3, age=27, gender="male", social_contacts=642),
    User(id=4, age=16, gender="male", social_contacts=123),
    User(id=5, age=35, gender="female", social_contacts=247),
]

input_collection_b = [
    User(id=5, age=35, gender="female", social_contacts=247),
    User(id=4, age=16, gender="male", social_contacts=123),
    User(id=7, age=32, gender="female", social_contacts=753),
]

def MAP_INTERSECTION(_, row: NamedTuple):
    yield(row.id, row)

def REDUCE_INTERSECTION(row: str, rows: Iterator[NamedTuple]):
  if len(rows) ==2:
    yield(rows[0], rows[1])

def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]

output = MapReduce(RECORDREADER, MAP_INTERSECTION, REDUCE_INTERSECTION)
output = list(output)
output

[(User(id=4, age=16, social_contacts=123, gender='male'),
  User(id=4, age=16, social_contacts=123, gender='male')),
 (User(id=5, age=35, social_contacts=247, gender='female'),
  User(id=5, age=35, social_contacts=247, gender='female'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [ ]:
input_collection_a = [
    User(id=1, age=54, gender="male", social_contacts=20),
    User(id=2, age=25, gender="female", social_contacts=240),
    User(id=3, age=27, gender="male", social_contacts=642),
    User(id=4, age=16, gender="male", social_contacts=123),
    User(id=5, age=35, gender="female", social_contacts=247),
]

input_collection_b = [
    User(id=5, age=35, gender="female", social_contacts=247),
    User(id=4, age=16, gender="male", social_contacts=123),
    User(id=7, age=32, gender="female", social_contacts=753),
]
# Группировка по пользователям
def MAP_DIFFERENCE(collection_id, user):
    yield (user, collection_id)


# Возвращает пользователей, входящих только в первую выборку (и не входящих во вторую)
def REDUCE_DIFFERENCE(user, collections):
    if 1 not in collections:
        yield (user)


# На первом месте стоит номер выборки
def RECORDREADER():
    return [(0, a) for a in input_collection_a] + [(1, b) for b in input_collection_b]


output = MapReduce(RECORDREADER, MAP_DIFFERENCE, REDUCE_DIFFERENCE)
output = list(output)
output

[User(id=1, age=54, social_contacts=20, gender='male'),
 User(id=2, age=25, social_contacts=240, gender='female'),
 User(id=3, age=27, social_contacts=642, gender='male')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [ ]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str
    country_id: int


users_collection = [
    User(id=1, age=54, gender="male", social_contacts=20, country_id=1),
    User(id=2, age=25, gender="female", social_contacts=240, country_id=2),
    User(id=3, age=27, gender="male", social_contacts=642, country_id=3),
    User(id=4, age=16, gender="male", social_contacts=123, country_id=3),
    User(id=5, age=35, gender="female", social_contacts=247, country_id=4),
]


class Country(NamedTuple):
    id: int
    name: str


countries_collection = [
    Country(id=1, name="Russia"),
    Country(id=2, name="China"),
    Country(id=3, name="Britain"),
    Country(id=4, name="USA"),
    Country(id=5, name="Japan")
]

# Группировка пользователей по id города
def MAP_JOIN(country_id, row):
    yield (country_id, row)


def REDUCE_JOIN(country_id, rows):
    users = []
    country = None

    for row in rows:
        if type(row) is User:
            users += [row]
        else:
            country = row

    for row in rows:
        if type(row) is User:
            # Возвращает тройки (a,b,c), где a - пользователь, b - id страны, c - сама страна
            yield (row, row.country_id, country)


# На первом месте стоит id страны, на втором - сама страна или пользователь
def RECORDREADER():
    return [(user.country_id, user) for user in users_collection] + [
        (country.id, country) for country in countries_collection
    ]


output = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
output = list(output)
joined_data = output
joined_data

[(User(id=1, age=54, social_contacts=20, gender='male', country_id=1),
  1,
  Country(id=1, name='Russia')),
 (User(id=2, age=25, social_contacts=240, gender='female', country_id=2),
  2,
  Country(id=2, name='China')),
 (User(id=3, age=27, social_contacts=642, gender='male', country_id=3),
  3,
  Country(id=3, name='Britain')),
 (User(id=4, age=16, social_contacts=123, gender='male', country_id=3),
  3,
  Country(id=3, name='Britain')),
 (User(id=5, age=35, social_contacts=247, gender='female', country_id=4),
  4,
  Country(id=4, name='USA'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [ ]:
# Группировка по id страны
def MAP_GROUP(country_id, user, city):
    yield (country_id, user)


# Подсчет количества пользователей в каждой стране
def REDUCE_GROUP(country_id, rows):
  yield f"country with id={country_id} has {len(rows)} user(s)"


def RECORDREADER():
    return [(country_id, user, city) for user, country_id, city in joined_data]


output = MapReduce(RECORDREADER, MAP_GROUP, REDUCE_GROUP)
output = list(output)
output

['country with id=1 has 1 user(s)',
 'country with id=2 has 1 user(s)',
 'country with id=3 has 2 user(s)',
 'country with id=4 has 1 user(s)']

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [ ]:
from typing import Iterator, Tuple

input_matrix = [
    (0, 0, 2), (0, 1, 3), (0, 2, 4),
    (1, 0, 1), (1, 1, 0), (1, 2, 2),
    (2, 0, 4), (2, 1, 1), (2, 2, 3),
]

input_vector = [
    (0, 1), (1, 2), (2, 3)
]


def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    grouped = {}
    for key, value in iterable:
        grouped[key] = grouped.get(key, []) + [value]
    return grouped.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )


def MAP_MATRIX_VECTOR(_, matrix_row):
    row, col, value = matrix_row

    for vec_col, vec_value in input_vector:
        if vec_col == col:
            yield (row, value * vec_value)


def REDUCE_MATRIX_VECTOR(row, values: Iterator[int]):
    yield (row, sum(values))


def RECORDREADER():
    return [(None, m) for m in input_matrix]


output = MapReduce(RECORDREADER, MAP_MATRIX_VECTOR, REDUCE_MATRIX_VECTOR)
output = list(output)
output

[(0, 20), (1, 7), (2, 15)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [ ]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [ ]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  # solution code that yield(k2,v2) pairs
  for i in range(I):
        k2 = (i, k)
        v2 = small_mat[i, j] * w
        yield (k2, v2)

def REDUCE(key, values):
  (i, k) = key
  # solution code that yield(k3,v3) pairs
  k3 = (i, k)
  v3 = sum(values)

  yield (k3, v3)

Проверьте своё решение

In [ ]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [ ]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [ ]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)


def RECORDREADER(): # сразу создаются две матрицы
    for i in range(I):
        for j in range(J):
            for k in range(K):
                yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))


def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)


def REDUCE(key, values):
    (i, k) = key

    # solution code that yield(k3,v3) pairs
    k3 = (i, k)
    v3 = sum(values)

    yield (k3, v3)

# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)


def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for (i, k), vw in reduce_output:
        mat[i, k] = vw
    return mat


np.allclose(reference_solution, asmatrix(solution))  # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [ ]:
maps = 2
reducers = 2


def INPUTFORMAT():
    global maps

    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

    # Разделение между вычислительными узлами происходит по строкам первой матрицы
    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))



def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)


def REDUCE(key, values):
    (i, k) = key
    k3 = (i, k)
    v3 = sum(values)

    yield (k3, v3)



partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]

# Соединение полученных произведений с разных вычислительных узлов
solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution += [element]

print(asmatrix(solution))
np.allclose(reference_solution, asmatrix(solution))  # should return true

240 key-value pairs were sent over a network.
[[0.81520692 0.88162159 0.90032296 0.28857278 0.26946805 0.98459533
  0.75941064 0.40073806 0.89168829 0.30337428 0.70174424 0.41807651
  0.05621778 0.72475418 0.47595954 0.32902394 0.28049652 0.88482583
  0.8674399  0.38296455 0.37786349 0.90616871 0.74925225 0.64218382
  0.3649871  0.36442826 0.630104   0.31259367 0.81532425 0.3279877
  0.61885258 0.3428322  0.42281932 0.79514718 0.12993166 0.79852618
  0.96307109 0.92805403 0.38085864 0.60323413]
 [0.86262706 1.08003236 1.29509445 1.0095528  1.10589525 1.15880175
  0.50170168 0.28302637 1.38592622 0.58941663 1.17739001 0.40661761
  0.46542704 0.89519349 0.24586385 0.90436225 0.91289776 1.37719451
  0.92993251 1.1222029  1.09482906 1.2824293  0.58568148 0.45803376
  1.23579614 1.10658557 0.29895987 0.98517118 1.00159722 0.433037
  0.491353   1.01685584 0.95493181 0.94769337 0.81568369 0.48710324
  0.97720405 1.0689345  0.27227473 0.86807685]]


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

Решение работать будет, поскольку генерируемые RECORREADER-ами элементы идентифицируются индексами матрицы.